# Parameters Tuning

In [1]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from mlforecast import MLForecast
from sklearn.metrics import mean_squared_error
from window_ops.rolling import rolling_mean
from tqdm import tqdm

path  = 'C:/PycharmProjects/pythonProject1/yjz/Extension for hts/JayCode/Data/'
data  = pd.read_pickle(f"{path}final_processed_data.pkl")#.reset_index(drop=True)
data["unique_id"] = pd.concat([pd.DataFrame({"unique_id": np.ones(1941)*(i+1)}) for i in tqdm(range(42840))]).reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████| 42840/42840 [00:09<00:00, 4735.82it/s]


## optuna

In [7]:
# define the goal function for optuna
nn1 = 1913-28*2
nn2 = 1913-28
DATA_COLS = ["y", "unique_id", "ds", 'sell_price', 'event_name_1','level','price_std', 'price_max', 'price_min', 'price_mean', 'price_std_dev',
             'price_momentum_1d', 'price_momentum_1m', 'price_momentum_1y',
             'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
             'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']
def objective(trial):
    train_df = train.loc[train["ds"] <= nn1].reset_index(drop=True)
    val_df   = train.loc[train["ds"] > nn1]
    val_df   = val_df.loc[val_df['ds'] < nn2].reset_index(drop=True)
    test_df  = val_df[val_df['ds'] >= nn2].reset_index(drop=True)
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": trial.suggest_int("n_estimators", 300, 1500),
        "random_state": 8,
        "verbosity": -1,
        "learning_rate": trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        "num_leaves": trial.suggest_int('num_leaves', 2, 256),
        "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 1, 100),
        "subsample": trial.suggest_uniform("subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_uniform('colsample_bytree', 0.3, 1.0),
        'num_threads': -1
    
    models = [lgb.LGBMRegressor(**params)]
    model = MLForecast(
            models=models,
            freq=1
        )
    model.fit(train_df,static_features = [])
    forecasts = model.predict(h=1,X_df = val_df[DATA_COLS[1:]])
    forecasts = forecasts.merge(val_df[["unique_id","ds","y"]], on = ["unique_id","ds"], how="left")
    rmse = mean_squared_error(forecasts["y"], forecasts["LGBMRegressor"], squared=False)
    
    return rmse

In [8]:
import warnings
warnings.filterwarnings("ignore")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-03-07 21:07:53,652] A new study created in memory with name: no-name-27ae2393-9c6c-444e-8c99-03bfb445a0fd


1
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1


[I 2025-03-07 21:24:02,816] Trial 0 finished with value: 150.74463996450936 and parameters: {'n_estimators': 930, 'learning_rate': 0.0016112910158499465, 'num_leaves': 99, 'min_data_in_leaf': 12, 'subsample': 0.7322324840407219, 'colsample_bytree': 0.8233825130977579}. Best is trial 0 with value: 150.74463996450936.


1
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1


[I 2025-03-07 21:42:13,497] Trial 1 finished with value: 148.67133980572288 and parameters: {'n_estimators': 995, 'learning_rate': 0.0015290321070419942, 'num_leaves': 133, 'min_data_in_leaf': 6, 'subsample': 0.4897141238349568, 'colsample_bytree': 0.8505094085277167}. Best is trial 1 with value: 148.67133980572288.


...

[I 2025-03-08 11:35:48,649] Trial 47 finished with value: 121.69056561664777 and parameters: {'n_estimators': 1207, 'learning_rate': 0.03166325481333476, 'num_leaves': 134, 'min_data_in_leaf': 38, 'subsample': 0.5642261594922485, 'colsample_bytree': 0.3070391991670757}. Best is trial 35 with value: 120.88363622654974.


1
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1


[I 2025-03-08 12:00:26,087] Trial 48 finished with value: 121.25562748611532 and parameters: {'n_estimators': 923, 'learning_rate': 0.04106111408991665, 'num_leaves': 171, 'min_data_in_leaf': 30, 'subsample': 0.4327721934850755, 'colsample_bytree': 0.4421656846304066}. Best is trial 35 with value: 120.88363622654974.


1
[LightGBM] [Warning] min_data_in_leaf is set=52, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=52
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1


[I 2025-03-08 12:24:13,768] Trial 49 finished with value: 121.39224346825316 and parameters: {'n_estimators': 1008, 'learning_rate': 0.022904272664593675, 'num_leaves': 203, 'min_data_in_leaf': 52, 'subsample': 0.39635468176208755, 'colsample_bytree': 0.6122356043313575}. Best is trial 35 with value: 120.88363622654974.


In [9]:
print(study.best_params)
print(study.best_value)
print(study.best_trial)

{'n_estimators': 952, 'learning_rate': 0.051106919899739436, 'num_leaves': 135, 'min_data_in_leaf': 38, 'subsample': 0.5955248360368148, 'colsample_bytree': 0.7277421110813926}
120.88363622654974
FrozenTrial(number=35, state=TrialState.COMPLETE, values=[120.88363622654974], datetime_start=datetime.datetime(2025, 3, 8, 7, 29, 53, 720315), datetime_complete=datetime.datetime(2025, 3, 8, 7, 47, 0, 172169), params={'n_estimators': 952, 'learning_rate': 0.051106919899739436, 'num_leaves': 135, 'min_data_in_leaf': 38, 'subsample': 0.5955248360368148, 'colsample_bytree': 0.7277421110813926}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1500, log=False, low=300, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=0.001, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'min_data_in_leaf': IntDistribution(high=100, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=Fa

## LGBM

In [4]:
#params_1 = study.best_params
'''
feats = ['y','unique_id', 'ds', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm',
       'tm_dw', 'tm_w_end', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'level', 'sell_price',
       'price_std', 'price_max', 'price_min', 'price_mean', 'price_std_dev',
       'price_momentum_1d', 'price_momentum_1m', 'price_momentum_1y']'''

In [24]:
%%time
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

lgb_model = lgb.train(params_2, lgb_train)
y_pred = lgb_model.predict(x_ts)
'''
mlf_lgbm = MLForecast(
    models=[lgb.LGBMRegressor(**params_2)],
    freq=1)
pred = []
for i in tqdm(range(28)):
    train = data[data['ds']<=1913+i]
    test  = data[data['ds']==1913+i+1]
    
    mlf_lgbm.fit(train,static_features=[])
    pred.append(mlf_lgbm.predict(h = 1, X_df = test[feats[1:]]))'''

Wall time: 13min 59s


"\nmlf_lgbm = MLForecast(\n    models=[lgb.LGBMRegressor(**params_2)],\n    freq=1)\npred = []\nfor i in tqdm(range(28)):\n    train = data[data['ds']<=1913+i]\n    test  = data[data['ds']==1913+i+1]\n    \n    mlf_lgbm.fit(train,static_features=[])\n    pred.append(mlf_lgbm.predict(h = 1, X_df = test[feats[1:]]))"

In [25]:
y_fitted = lgb_model.predict(x_tr)

In [27]:
len(y_fitted)

81952920

In [28]:
81952920/1913

42840.0

In [32]:
pd.DataFrame({'lgbm_base':y_pred}).to_pickle("h1LGBMbase.pkl")
pd.DataFrame({"lgbm_fitted":y_fitted}).to_pickle("h1LGBMfitt.pkl")

## recursive

In [3]:
data

,unique_id,ds,y,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,...,price_momentum_1d,price_momentum_1m,price_momentum_1y,y_lags_1,y_lags_2,y_lags_3,y_lags_4,y_lags_5,y_lags_6,y_lags_7
0,1.0,1,32631,29,4,1,0,5,5,1,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2,31749,30,4,1,0,5,6,1,...,0.0,0.0,0.000000,32631.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,3,23783,31,5,1,0,5,0,0,...,0.0,0.0,0.000000,31749.0,32631.0,NaN,NaN,NaN,NaN,NaN
3,1.0,4,25412,1,5,2,0,1,1,0,...,0.0,0.0,0.137056,23783.0,31749.0,32631.0,NaN,NaN,NaN,NaN
4,1.0,5,19146,2,5,2,0,1,2,0,...,0.0,0.0,0.000000,25412.0,23783.0,31749.0,32631.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83152435,42840.0,1937,0,18,20,5,5,3,2,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
83152436,42840.0,1938,0,19,20,5,5,3,3,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
83152437,42840.0,1939,0,20,20,5,5,3,4,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0
83152438,42840.0,1940,0,21,20,5,5,3,5,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [2]:
%%time
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

params_2 = {'n_estimators': 952, 
            'learning_rate': 0.051106919899739436, 
            'num_leaves': 135, 
            'min_data_in_leaf': 38, 
            'subsample': 0.5955248360368148, 
            'colsample_bytree': 0.7277421110813926,
            "objective": "regression",
            "metric": "rmse", 
            "random_state": 8,
            "verbosity": -1,
            'num_threads': -1}

y = data[['ds','y']]
x = data.drop(columns=['y'])
y_p = []
for i in tqdm(range(28)):
    xr_ts = x[x['ds']==1913+i+1]
    lgb_tr = lgb.Dataset(x[x['ds']<=(1913+i)], y[y['ds']<=(1913+i)]['y'])
    lgb_m = lgb.train(params_2, lgb_tr)
    y_p.append(pd.DataFrame({f"{1913+i+1}":lgb_m.predict(xr_ts)}))
    del lgb_tr
pd.concat(y_p,axis=1).to_pickle("h1_LGBM_base_recursive.pkl")

100%|███████████████████████████████████████████████████████████████████████| 28/28 [10:14:34<00:00, 1316.95s/it]

Wall time: 10h 14min 51s


In [9]:
lgb_ = pd.concat(y_p,axis=1)
lgb222 = pd.concat([pd.DataFrame({'LGBMRegressor': lgb_.iloc[i,:] }) for i in tqdm(range(42840))])
lgb222.to_pickle("h1_LGBMbase_recursive.pkl")

100%|████████████████████████████████████████████████████████████████████| 42840/42840 [00:13<00:00, 3247.76it/s]


## obtain fitted

In [4]:
%%time
params_2 = {'n_estimators': 952, 
            'learning_rate': 0.051106919899739436, 
            'num_leaves': 135, 
            'min_data_in_leaf': 38, 
            'subsample': 0.5955248360368148, 
            'colsample_bytree': 0.7277421110813926,
            "objective": "regression",
            "metric": "rmse", 
            "random_state": 8,
            "verbosity": -1,
            'num_threads': -1}
xtr,xts = x[x['ds']<=1913],x[x['ds']>1913]
ytr,yts = y[y['ds']<=1913]['y'],y[y['ds']>1913]['y']
lgbtr = lgb.Dataset(xtr,ytr)
lgb2 = lgb.train(params_2, lgbtr)
yfitt = lgb2.predict(xtr)
pred2 = lgb2.predict(xts)
print("finished train")

finished train
Wall time: 27min 35s


In [7]:
pd.DataFrame({"LGBMRegressor":yfitt}).to_pickle("h1_LGBMfitt.pkl")

In [8]:
pd.DataFrame({"LGBMRegressor":pred2}).to_pickle("h1_LGBMbase_norecursive.pkl")